In [96]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)  # отобразить все колонки
pd.set_option('display.max_rows', None)

In [97]:
df = pd.read_csv('../data/_data.with_new_cols.csv')

/tmp/ipykernel_77411/2416377651.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/_data.with_new_cols.csv')


In [98]:
df[['floor', 'n_of_floors', 'build_type']] = df['Дом'].str.extract(r'(\d+)[/\\](\d+),?\s*([А-Яа-яA-Za-z\s]+)?')  # Добавила 3 колонки

In [99]:
df = df.drop_duplicates(
    subset=['Адрес', 'Общая площадь', 'rooms_qty', 'floor', 'n_of_floors', 'Ремонт'],
    keep='first'
).copy() # Убрали дублирующие строки 454


In [100]:
# Ремонт, Можно с детьми/животными, Название ЖК чистое, Парковка, Station, Line, AdmArea, District - заполнение NaN строкой "Данные отсуствуют"

df["Ремонт"] = df["Ремонт"].fillna("Данные отсутствуют")
df["Можно с детьми/животными"] = df["Можно с детьми/животными"].fillna("Данные отсутствуют")
df["Название ЖК чистое"] = df["Название ЖК чистое"].fillna("Данные отсутствуют")
df["Парковка"] = df["Парковка"].fillna("Данные отсутствуют")
df["Station"] = df["Station"].fillna("Данные отсутствуют")
df["Line"] = df["Line"].fillna("Данные отсутствуют")
df["AdmArea"] = df["AdmArea"].fillna("Данные отсутствуют")
df["District"] = df["District"].fillna("Данные отсутствуют")

In [101]:
# Балкон, Окна, Санузел, Телефоны, Описание - заполнение NaN, None значением False(0), значением True(1) - заполненные значения

df["Балкон"] = df["Балкон"].notna().astype(int)
df["Окна"] = df["Окна"].notna().astype(int)
df["Санузел"] = df["Санузел"].notna().astype(int)
df["Телефоны"] = df["Телефоны"].notna().astype(int)
df["Описание"] = df["Описание"].notna().astype(int)

In [102]:
# Создаем новые колонки "Мебель в комнатax" и "Мебель на кухне" из колонки "Дополнительно" - заполняем значениями True/False (0, 1)

df["Мебель в комнатах"] = df["Дополнительно"].str.contains("Мебель в комнатах", na=False).astype(int)
df["Мебель на кухне"] = df["Дополнительно"].str.contains("Мебель на кухне", na=False).astype(int)

In [103]:
df.fillna({'Высота потолков, м': df['Высота потолков, м'].median()}, inplace=True) # Заполнили медианой пропуски

In [104]:
df['Лифт'] = (
    df['Лифт']
    .astype(str)          
    .str.lower().str.strip()
    .apply(lambda x: 0 if x == 'nan' else 1)
)  # Значения в колонке Лифт заменены на 1 - есть значение / - nan

In [105]:
df['Мусоропровод'] = df['Мусоропровод'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'
df['room_type'] = df['room_type'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'
df['build_type'] = df['build_type'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'

In [106]:
df['Город'] = df['Город'].apply(lambda x: 1 if x == 'Москва' else 0) # Закодировали 'Город' 1/0

In [107]:
df[['street', 'bld.']] = df['Адрес'].str.split(',',expand=True).iloc[:, 1:3] # Выделили адрес в две колонки streen, bld.

In [108]:
# Оставляем в датафрейме только предложения со стоимостью, указанной в рублях
df = df[df['Цена'].str.contains("руб", case=False, na=True)]

In [109]:
# Разделение параметра 'Путь до метро' на 2 - время (int) и досягаемость (пешком/на машине)
df[['Время до метро', 'Досягаемость']] = df['Путь до метро'].str.extract(r'(\d+)\s*мин\s*(.*)')
df['Время до метро'] = pd.to_numeric(df['Время до метро'], errors='coerce').astype('Int64')

In [110]:
#df['Время до метро'] = df['Время до метро'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'

In [111]:
df = df.drop(['Unnamed: 0','Количество комнат', 'Дом', 'Название ЖК', 'Тип', 'Метро', 'Адрес', 'Площадь, м2', 'Цена', 'Площадь комнат, м2',
              'Дополнительно', 'Серия дома', 'Ссылка на объявление', 'ЖК год', 'Станция', 'Путь до метро'], axis=1)

In [126]:
missing_counts = df.isna().sum()
unique_counts = df.nunique(dropna=False)
df_types = df.dtypes
summary = pd.DataFrame({
    'Пропуски (шт.)': missing_counts,
    'Уникальных значений (вкл. NaN)': unique_counts,
    'Типы данных': df_types
})
summary

,Пропуски (шт.),Уникальных значений (вкл. NaN),Типы данных
ID,0,19254,int64
Parking,0,6,int64
Phone,0,1,int64
Description,0,1,int64
Renovation_type,0,5,int64
Balcony,0,2,int64
Windows,0,2,int64
WC,0,2,int64
Chld_pets_allowed,0,4,int64
Lift,0,2,int64


In [113]:
df.fillna({'Время до метро': df['Время до метро'].median()}, inplace=True) # Заполнили медианой пропуски

In [114]:
df['Досягаемость'] = df['Досягаемость'].fillna('Данные отсутствуют') # Заменили NaN на 'Данные отсутствуют'

In [115]:
# добавляем новую фичу - Цена за метр
df['Цена за метр'] = df['Цена за месяц'] / df['Общая площадь']

In [116]:
# Для нормализации данных по колонкам Цена за месяц и Общая площадь применено Логарифмирование (log-transform)
# Тк это особенно полезно для MAPE, уменьшает разброс относительных ошибок, делает MAPE более устойчивым

df["Цена за месяц"] = np.log1p(df["Цена за месяц"]) # обратное после предсказания np.expm1(df["Цена за месяц"])
df["Общая площадь"] = np.log1p(df["Общая площадь"])
df["Цена за метр"] = np.log1p(df["Цена за метр"])

In [117]:
def convert_to_meters(value):
    try:
        num = float(value)
        # Если число меньше 10 (предполагаем, что это метры с дробной частью)
        # Или если число больше 100 (предполагаем, что это сантиметры)
        if 10 < num < 100:  # Значения больше 100 считаем сантиметрами
            return num / 10
        elif num > 100:
            return num / 100
        else:  # Остальные считаем метрами
            return num
    except:
        return np.nan  # Для некорректных значений

In [118]:
# Применяем функцию convert_to_meters
df['Высота корректная, м'] = df['Высота потолков, м'].apply(convert_to_meters)

In [119]:
# Оставляем в датафрейме только предложения с высотой потолка >= 2.4 м
df = df[df['Высота корректная, м'] >= 2.4]

In [120]:
df = df.drop(['Высота потолков, м'], axis=1) # удалили изначальную колонку с Высотой потолка

In [121]:
df[['floor', 'n_of_floors']] = df[['floor', 'n_of_floors']].astype(int)

In [122]:
from sklearn.preprocessing import LabelEncoder

cols = [
    'Парковка',
    'Ремонт',
    'Можно с детьми/животными',
    'Мусоропровод',
    'Название ЖК чистое',
    'room_type',
    'Station',
    'Line',
    'AdmArea',
    'District',
    'build_type',
    'street',
    'bld.',
    'Досягаемость'
    ]

le = LabelEncoder()
df['Парковка'] = le.fit_transform(df['Парковка'])

le2 = LabelEncoder()
df['Ремонт'] = le2.fit_transform(df['Ремонт'])

le3 = LabelEncoder()
df['Можно с детьми/животными'] = le3.fit_transform(df['Можно с детьми/животными'])

le4 = LabelEncoder()
df['Мусоропровод'] = le4.fit_transform(df['Мусоропровод'])

le5 = LabelEncoder()
df['Название ЖК чистое'] = le5.fit_transform(df['Название ЖК чистое'])

le6 = LabelEncoder()
df['room_type'] = le6.fit_transform(df['room_type'])

le7 = LabelEncoder()
df['Station'] = le7.fit_transform(df['Station'])

le8 = LabelEncoder()
df['Line'] = le8.fit_transform(df['Line'])

le9 = LabelEncoder()
df['AdmArea'] = le9.fit_transform(df['AdmArea'])

le10 = LabelEncoder()
df['District'] = le10.fit_transform(df['District'])

le11 = LabelEncoder()
df['build_type'] = le11.fit_transform(df['build_type'])

le12 = LabelEncoder()
df['street'] = le12.fit_transform(df['street'])

le13 = LabelEncoder()
df['bld.'] = le13.fit_transform(df['bld.'])

le14 = LabelEncoder()
df['Досягаемость'] = le14.fit_transform(df['Досягаемость'])

In [123]:
df.rename(columns={'ID  объявления' :'ID',   'Парковка' :'Parking',  'Телефоны' :'Phone', 'Описание' :'Description', 'Ремонт' :'Renovation_type',
           'Балкон' :'Balcony', 'Окна' :'Windows',  'Санузел' :'WC',    'Можно с детьми/животными' :'Chld_pets_allowed',    'Лифт' :'Lift',
           'Мусоропровод' :'Refuse_chute',  'Город' :'City',    'Цена за месяц' :'Price_per_month', 'Название ЖК чистое' :'Res_complex_name',
           'Общая площадь' :'Total_sqr_mtr',    'rooms_qty' :'Rooms_qty',   'room_type' :'Room_type',   'floor' :'Floor',   
           'n_of_floors' :'N_of_floor', 'build_type' :'Bld_type',   'Мебель в комнатах' :'Furniture_rooms', 
           'Мебель на кухне' :'Furniture_kitchen',  'street' :'Street', 'bld.' :'Bld.', 'Время до метро' :'Time_to_subway', 
           'Досягаемость' :'Accessibility', 'Высота корректная, м' :'Ceiling_height', 'Цена за метр': 'price_per_m'}, inplace=True)

In [124]:
df.head()

,ID,Parking,Phone,Description,Renovation_type,Balcony,Windows,WC,Chld_pets_allowed,Lift,Refuse_chute,City,Price_per_month,Res_complex_name,Total_sqr_mtr,Rooms_qty,Room_type,Station,Line,AdmArea,District,Floor,N_of_floor,Bld_type,Furniture_rooms,Furniture_kitchen,Street,Bld.,Time_to_subway,Accessibility,price_per_m,Ceiling_height
0,271271157,5,1,1,2,0,0,0,2,1,0,1,13.122365,359,5.303305,4,0,208,6,9,36,5,16,5,1,1,1563,872,9,2,7.824446,3.0
1,271634126,5,1,1,2,0,1,1,1,1,2,1,13.122365,359,5.293305,4,0,208,6,9,36,5,16,4,1,1,1563,872,8,2,7.834492,3.5
2,271173086,5,1,1,3,0,1,1,1,1,1,1,13.122365,359,5.303305,4,2,208,6,9,36,5,16,1,1,1,1563,872,7,2,7.824446,3.2
3,272197456,5,1,1,3,0,1,1,3,1,1,1,12.899222,170,5.141664,4,2,208,6,9,36,5,6,1,1,1,1177,721,3,2,7.763846,3.2
4,273614615,0,1,1,3,0,1,1,0,1,0,1,12.323860,72,4.077537,2,0,10,6,9,36,12,26,6,1,1,1563,396,7,2,8.263670,3.9


In [127]:
df.to_csv("comets_gr2_data.csv", index=False)